# Gallery for the Overview Paper

The purpose of this notebook is to build a nice gallery of object images for the overview paper.

For future reference: The notebook must be run from https://jupyter-dev.nersc.gov with the following (approximate) activation script:

```bash
#!/bin/bash                                                                                                           
version=$1                                                                                                            
connection_file=$2                                                                                                    

desiconda_version=20170818-1.1.12-img                                                                                 
module use /global/common/${NERSC_HOST}/contrib/desi/desiconda/$desiconda_version/modulefiles                         
module load desiconda                                                                                                 

export LEGACY_SURVEY_DIR=/global/cscratch1/sd/dstn/dr6plus                                                            
export LEGACYPIPE_DIR=$SCRATCH/repos/legacypipe                                                                       

export PATH=$LEGACYPIPE_DIR/bin:${PATH}                                                                               
export PATH=$SCRATCH//repos/build/bin:$PATH                                                                           
export PYTHONPATH=$LEGACYPIPE_DIR/py:${PYTHONPATH}                                                                    
export PYTHONPATH=$SCRATCH/repos/build/lib/python3.5/site-packages:$PYTHONPATH                                        

module use $LEGACYPIPE_DIR/bin/modulefiles/cori                                                                       
module load dust                                                                                                      

exec python -m ipykernel -f $connection_file
```

Some neat objects:
* [Bow shock](http://legacysurvey.org/viewer?ra=325.6872&dec=1.0032&zoom=14&layer=decals-dr5)
* [Abell 383](http://legacysurvey.org/viewer?ra=42.0141&dec=-3.5291&zoom=15&layer=decals-dr5)
* [SDSS/C4-2010 Galaxy Cluster](http://legacysurvey.org/viewer?ra=29.0707&dec=1.0510&zoom=13&layer=decals-dr5)
* [NGC2874 Group](http://legacysurvey.org/viewer?ra=141.4373&dec=11.4284&zoom=13&layer=decals-dr5)
* [UGC10321 Group](http://legacysurvey.org/viewer?ra=244.5280&dec=21.5591&zoom=14&layer=decals-dr5)
* [NGC6742 (PN)](http://legacysurvey.org/viewer/?layer=decals-dr6&ra=284.83291667&dec=48.46527778)

### Imports and paths

In [1]:
import os, sys
import shutil, time, warnings
from contextlib import redirect_stdout
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt

In [2]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column, vstack
from astropy.io import ascii
from PIL import Image, ImageDraw, ImageFont
#from astrometry.util.starutil_numpy import hmsstring2ra

In [3]:
from astrometry.util.util import Tan
from astrometry.util.fits import merge_tables
from legacypipe.survey import LegacySurveyData
from legacypipe.runbrick import run_brick

In [4]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [5]:
%matplotlib inline

### Preliminaries

Define the data release and the various output directories.

In [6]:
PIXSCALE = 0.262
dr5_dir = '/global/cscratch1/sd/desiproc/dr5-new'
dr6_dir = '/global/cscratch1/sd/dstn/dr6plus'

In [7]:
figdir = os.path.join( os.getenv('SCRATCH'), 'overview-paper')
figfile = os.path.join(figdir, 'gallery.fits')

In [8]:
jpgdir = os.path.join(figdir, 'jpg')
if not os.path.isdir(jpgdir):
    os.mkdir(jpgdir)

In [9]:
pngdir = os.path.join(figdir, 'png')
if not os.path.isdir(pngdir):
    os.mkdir(pngdir)

#### Build a sample with the objects of interest.

In [10]:
nobj = 3

In [11]:
cat = Table()
cat.add_column(Column(name='name', length=nobj, dtype='U20'))
cat.add_column(Column(name='nicename', length=nobj, dtype='U30'))
cat.add_column(Column(name='ra', unit='deg', length=nobj, dtype='f8'))
cat.add_column(Column(name='dec', unit='deg', length=nobj, dtype='f8'))
cat.add_column(Column(name='diam', unit='arcsec', length=nobj, dtype='f4'))
cat.add_column(Column(name='dr', length=nobj, dtype='U2'))

In [12]:
cat['name'] = ('C4-2010', 'NGC2874', 'NGC6742')
cat['nicename'] = ('C4 Cluster 2010', 'NGC2874 Galaxy Group', 'NGC6742 Planetary Nebula')
cat['ra'] = (29.070641492, 141.44215000, 284.83291667)
cat['dec'] = (1.050816667, 11.43696000, 48.46527778)
cat['diam'] = (180, 120, 30) # arcsec
cat['dr'] = ('dr5', 'dr5', 'dr6')
cat

name,nicename,ra,dec,diam,dr
str7,str24,float64,float64,int64,str3
C4-2010,C4 Cluster 2010,29.070641492,1.050816667,180,dr5
NGC2874,NGC2874 Galaxy Group,141.44215,11.43696,120,dr5
NGC6742,NGC6742 Planetary Nebula,284.83291667,48.46527778,30,dr6


### Generate (find) the sample of objects in the DR6 footprint.

In [13]:
def init_survey(dr='dr5'):
    survey = ''
    try:
        del survey
    except:
        pass
    
    if dr == 'dr5':
        survey = LegacySurveyData(
            #survey_dir='/global/cscratch1/sd/dstn/dr5-new-sky',
            survey_dir='/global/cscratch1/sd/desiproc/dr7',
            #survey_dir='/global/project/projectdirs/cosmo/work/legacysurvey/dr5',
            #cache_dir='/global/cscratch1/sd/desiproc/dr5-calib',
            #survey_dir='/global/cscratch1/sd/desiproc/dr5-calib',
            #cache_dir='/global/cscratch1/sd/desiproc/dr7/calib',
            output_dir=figdir)
    elif dr == 'dr6':
        survey = LegacySurveyData(
            survey_dir='/global/cscratch1/sd/dstn/dr6plus',
            output_dir=figdir)
    print('!!!!!!!!!!!!!!!!!!', survey.survey_dir)
        
    #survey = LegacySurveyData(cache_dir='/global/cscratch1/sd/desiproc/dr7/calib/decam')
    #survey.output_dir = figdir
    
    return survey

In [14]:
def simple_wcs(obj):
    """Build a simple WCS object for a single object."""
    size = np.rint(obj['diam'] / PIXSCALE).astype('int') # [pixels]
    wcs = Tan(obj['ra'], obj['dec'], size/2+0.5, size/2+0.5,
                 -PIXSCALE/3600.0, 0.0, 0.0, PIXSCALE/3600.0, 
                 float(size), float(size))
    return wcs

In [15]:
def _build_sample_one(args):
    """Wrapper function for the multiprocessing."""
    return build_sample_one(*args)

In [16]:
def build_sample_one(obj, verbose=False):
    """Wrapper function to find overlapping grz CCDs for a given object.
    
    """
    print('!!!!', obj['dr'])
    survey = init_survey(dr=obj['dr'])
    
    print('Working on {}...'.format(obj['name']))
    wcs = simple_wcs(obj)
    try:
        ccds = survey.ccds_touching_wcs(wcs) # , ccdrad=2*diam/3600)
    except:
        return None
    
    if ccds:
        # Is there 3-band coverage?
        if 'g' in ccds.filter and 'r' in ccds.filter and 'z' in ccds.filter:
            if verbose:
                print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Diameter={:.4f} arcmin'.format(
                        obj['name'], len(ccds), obj['ra'], obj['dec'], obj['diam'] / 60.0))
            return obj
    return None

In [17]:
def build_sample(cat, use_nproc=nproc):
    """Build the full sample with grz coverage in DR6."""

    sampleargs = list()
    for cc in cat:
        sampleargs.append( (cc, True) ) # the False refers to verbose=False

    if use_nproc > 1:
        p = multiprocessing.Pool(nproc)
        result = p.map(_build_sample_one, sampleargs)
        p.close()
    else:
        result = list()
        for args in sampleargs:
            result.append(_build_sample_one(args))

    # Remove non-matching objects and write out the sample
    outcat = vstack(list(filter(None, result)))
    print('Found {}/{} objects in the DR5+6 footprint.'.format(len(outcat), len(cat)))
    
    return outcat

In [18]:
sample = build_sample(cat, use_nproc=1)

!!!! dr5
!!!!!!!!!!!!!!!!!! /global/cscratch1/sd/desiproc/dr7
Working on C4-2010...
Searching /global/cscratch1/sd/desiproc/dr7/survey-ccds-dr7-uncut.kd.fits
2348 CCDs within 1.0 deg of RA,Dec (29.071, 1.051)
For C4-2010 found 51 CCDs, RA = 29.07064, Dec = 1.05082, Diameter=3.0000 arcmin
!!!! dr5
!!!!!!!!!!!!!!!!!! /global/cscratch1/sd/desiproc/dr7
Working on NGC2874...
Searching /global/cscratch1/sd/desiproc/dr7/survey-ccds-dr7-uncut.kd.fits
2022 CCDs within 1.0 deg of RA,Dec (141.442, 11.437)
For NGC2874 found 35 CCDs, RA = 141.44215, Dec = 11.43696, Diameter=2.0000 arcmin
!!!! dr6
!!!!!!!!!!!!!!!!!! /global/cscratch1/sd/dstn/dr6plus
Working on NGC6742...
Searching /global/cscratch1/sd/dstn/dr6plus/survey-ccds-dr6plus.kd.fits
162 CCDs within 1.0 deg of RA,Dec (284.833, 48.465)
For NGC6742 found 7 CCDs, RA = 284.83292, Dec = 48.46528, Diameter=0.5000 arcmin
Found 3/3 objects in the DR5+6 footprint.


In [19]:
print('Writing {}'.format(figfile))
sample.write(figfile, overwrite=True)

Writing /global/cscratch1/sd/ioannis/overview-paper/gallery.fits


In [20]:
sample

name,nicename,ra,dec,diam,dr
str28,str96,float64,float64,int64,str12
C4-2010,C4 Cluster 2010,29.070641492,1.050816667,180,dr5
NGC2874,NGC2874 Galaxy Group,141.44215,11.43696,120,dr5
NGC6742,NGC6742 Planetary Nebula,284.83291667,48.46527778,30,dr6


### Generate the color mosaics for each object.

In [21]:
def custom_brickname(obj, prefix='custom-'): 
    brickname = 'custom-{:06d}{}{:05d}'.format(
        int(1000*obj['ra']), 'm' if obj['dec'] < 0 else 'p', 
        int(1000*np.abs(obj['dec'])))
    return brickname

In [22]:
def make_coadds_one(obj, scale=PIXSCALE, clobber=False):
    name = obj['name']
    jpgfile = os.path.join(jpgdir, '{}.jpg'.format(name))
    if os.path.isfile(jpgfile) and not clobber:
        print('File {} exists...skipping.'.format(jpgfile))
    else:
        diam = obj['diam'] # [arcsec]
        
        size = np.rint(diam / scale).astype('int') # [pixels]
        print('Generating mosaic for {} with width={} pixels.'.format(name, size))
        
        survey = init_survey(dr=obj['dr'])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            run_brick(None, survey, radec=(obj['ra'], obj['dec']), pixscale=scale, 
                      width=size, height=size, stages=['image_coadds'], splinesky=True,
                      early_coadds=True, pixPsf=True, hybridPsf=True, normalizePsf=True,
                      write_pickles=False, depth_cut=False, apodize=True, threads=nproc,
                      do_calibs=False, ceres=False)
            
        sys.stdout.flush()    
        brickname = custom_brickname(obj, prefix='custom-')
        _jpgfile = os.path.join(survey.output_dir, 'coadd', 'cus', brickname, 
                               'legacysurvey-{}-image.jpg'.format(brickname))
        shutil.copy(_jpgfile, jpgfile)
        shutil.rmtree(os.path.join(survey.output_dir, 'coadd'))

In [23]:
def make_coadds(sample, clobber=False):
    for obj in sample:
        make_coadds_one(obj, clobber=clobber)

In [ ]:
#make_coadds_one(sample[0], clobber=True)

In [ ]:
coaddslogfile = os.path.join(figdir, 'make-coadds.log')
print('Generating the coadds.')
print('Logging to {}'.format(coaddslogfile))
t0 = time.time()
with open(coaddslogfile, 'w') as log:
    with redirect_stdout(log):
        make_coadds(sample, clobber=True)
print('Total time = {:.3f} minutes.'.format((time.time() - t0) / 60))

Generating the coadds.
Logging to /global/cscratch1/sd/ioannis/overview-paper/make-coadds.log


In [ ]:
stop

### Add labels and a scale bar.

In [ ]:
barlen = np.round(60.0 / PIXSCALE).astype('int')
fonttype = os.path.join(gallerydir, 'Georgia.ttf')

In [ ]:
def _add_labels_one(args):
    """Wrapper function for the multiprocessing."""
    return add_labels_one(*args)

In [ ]:
def add_labels_one(obj, verbose=False):
    name = get_name(obj)
    nicename = get_name(obj, nice=True)
    
    jpgfile = os.path.join(jpgdir, '{}.jpg'.format(name))
    pngfile = os.path.join(pngdir, '{}.png'.format(name))
    thumbfile = os.path.join(pngdir, 'thumb-{}.png'.format(name))
        
    im = Image.open(jpgfile)
    sz = im.size
    fntsize = np.round(sz[0]/28).astype('int')
    width = np.round(sz[0]/175).astype('int')
    font = ImageFont.truetype(fonttype, size=fntsize)
    draw = ImageDraw.Draw(im)
    # Label the object name--
    draw.text((0+fntsize*2, 0+fntsize*2), nicename, font=font)
    # Add a scale bar--
    x0, x1, yy = sz[1]-fntsize*2-barlen, sz[1]-fntsize*2, sz[0]-fntsize*2
    draw.line((x0, yy, x1, yy), fill='white', width=width)
    im.save(pngfile)    
        
    # Generate a thumbnail
    cmd = '/usr/bin/convert -thumbnail 300x300 {} {}'.format(pngfile, thumbfile)
    os.system(cmd)

In [ ]:
def add_labels(sample):
    labelargs = list()
    for obj in sample:
        labelargs.append((obj, False))

    if nproc > 1:
        p = multiprocessing.Pool(nproc)
        res = p.map(_add_labels_one, labelargs)
        p.close()
    else:
        for args in labelargs:
            res = _add_labels_one(args)

In [ ]:
%time add_labels(sample)

### Finally, assemble the webpage of good and rejected gallery images.

To test the webpage before release, do
  * rsync -auvP /global/cscratch1/sd/ioannis/dr6/gallery/png /global/project/projectdirs/cosmo/www/temp/ioannis/dr6/gallery/
  * rsync -auvP /global/cscratch1/sd/ioannis/dr6/gallery/*.html /global/project/projectdirs/cosmo/www/temp/ioannis/dr6/gallery/

In [ ]:
def get_type(hubble):
    """Convert Hubble type to numerical type, for sorting purposes."""
    numtype = {
        'E': 0,
        'E-S0': 1,
        
        'S0': 2,
        'S0-a': 3,
        
        'Sa': 4,
        'SBa': 4,
        'SABa': 4,

        'Sab': 5,
        'SBab': 5,
        
        'Sb': 6,
        'SABb': 6,
        'SBb': 6,

        'Sbc': 7,
        
        'Sc': 8,
        'SABc': 8,
        'SBc': 8,
        
        'Scd': 9,
        'SBcd': 9,
        
        'Sd': 10,
        
        'Sm': 11,
        'SBm': 11,
        
        'I': 12,
        'IAB': 12,
        'IB': 12,
        
        '0': -1
    }
    return np.array([numtype[hh] for hh in hubble])

In [ ]:
reject = ['ngc3587', 'ngc6832', 'ngc5982', 'ngc2832', 'ngc2340', 'ngc5195',
          'ngc5308', 'ngc4346', 'ngc4036', 'ngc2681', 'ngc3718', 'ngc5377',
          'ngc2146', 'ngc3126', 'ngc2841', 'ngc2683', 'ngc4217', 'ngc4357',
          'ngc5055', 'ngc4100', 'ngc5879', 'ngc5297', 'ngc4605', 'ngc6015',
          'ngc4144', 'ngc3733', 'ngc3079', 'ngc3198', 'ngc3430', 'ngc3877',
          'ngc4062', 'ngc4631', 'ngc4656_ned01', 'ngc4395']
toss = np.zeros(len(sample), dtype=bool)
name = get_name(sample)
for ii, nn in enumerate(name):
    for rej in np.atleast_1d(reject):
        toss[ii] = rej in nn.lower()
        if toss[ii]:
            break
print('Rejecting {} objects.'.format(np.sum(toss)))
pngkeep = sample[~toss]
if np.sum(toss) > 0:
    pngrej = sample[toss]
else:
    pngrej = []

In [ ]:
htmlfile = os.path.join(gallerydir, 'index.html')
htmlfile_reject = os.path.join(gallerydir, 'index-reject.html')
baseurl = 'http://legacysurvey.org/viewer-dev'

In [ ]:
def html_rows(pngkeep, nperrow=4):
    nrow = np.ceil(len(pngkeep) / nperrow).astype('int')
    pngsplit = list()
    for ii in range(nrow):
        i1 = nperrow*ii
        i2 = nperrow*(ii+1)
        if i2 > len(pngkeep):
            i2 = len(pngkeep)
        pngsplit.append(pngkeep[i1:i2])
    #pngsplit = np.array_split(pngkeep, nrow)
    print('Splitting the sample into {} rows with {} mosaics per row.'.format(nrow, nperrow))

    html.write('<table class="ls-gallery">\n')
    html.write('<tbody>\n')
    for pngrow in pngsplit:
        html.write('<tr>\n')
        for obj in pngrow:
            name = get_name(obj)
            nicename = get_name(obj, nice=True)
            pngfile = os.path.join('png', '{}.png'.format(name))
            thumbfile = os.path.join('png', 'thumb-{}.png'.format(name))
            img = 'src="{}" alt="{}"'.format(thumbfile, nicename)
            #img = 'class="ls-gallery" src="{}" alt="{}"'.format(thumbfile, nicename)
            html.write('<td><a href="{}"><img {}></a></td>\n'.format(pngfile, img))
        html.write('</tr>\n')
        html.write('<tr>\n')
        for obj in pngrow:
            nicename = get_name(obj, nice=True)
            href = '{}/?layer=decals-{}&ra={:.8f}&dec={:.8f}&zoom=12'.format(baseurl, dr, obj['ra'], obj['dec'])
            html.write('<td><a href="{}" target="_blank">{}</a></td>\n'.format(href, nicename))
        html.write('</tr>\n')
    html.write('</tbody>\n')            
    html.write('</table>\n')

In [ ]:
objtype = ma.getdata(pngkeep['type'])
hubbletype = get_type(ma.getdata(pngkeep['hubble']))

In [ ]:
with open(htmlfile, 'w') as html:
    html.write('<html><head>\n')
    html.write('<style type="text/css">\n')
    html.write('table.ls-gallery {width: 90%;}\n')
    #html.write('img.ls-gallery {display: block;}\n')
    #html.write('td.ls-gallery {width: 100%; height: auto}\n')
    #html.write('td.ls-gallery {width: 100%; word-wrap: break-word;}\n')
    html.write('p.ls-gallery {width: 80%;}\n')
    html.write('</style>\n')
    html.write('</head><body>\n')
    html.write('<h1>DR6 Image Gallery</h1>\n')
    html.write("""<p class="ls-gallery">This gallery highlights the exquisite image quality and diversity 
    of objects observed by the Legacy Survey, including planetary nebulae, globular clusters, and 
    large, nearby galaxies.  Each thumbnail links to a larger image while the object name below each 
    thumbnail links to the 
    <a href="http://legacysurvey.org/viewer">Sky Viewer</a>.  For reference, the horizontal white bar in 
    the lower-right corner of each image represents one arcminute.</p>\n""")
    html.write("""<p>We gratefully acknowledge the <a href="https://github.com/mattiaverga/OpenNGC" target="_blank">
    OpenNGC</a> catalog created by Mattia Verga, which was used to generate this sample.</p>\n""")
    html.write("""<p>For more eye candy, please visit the gallery of galaxy groups highlighted in the 
    <a href="http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr5/gallery/">DR5 Gallery.</a></p>\n""")
    
    # Split by object type
    
    html.write('<h2>Planetary Nebulae, Open Clusters, and Globular Clusters</h2>\n')
    these = np.logical_or( np.logical_or(objtype == 'PN', objtype == 'OCl'), objtype == 'GCl' )
    srt = np.argsort(objtype[these])[::-1]
    html_rows(pngkeep[these][srt])
    html.write('<br />\n')
    
    html.write('<h2>Spheroidal & Elliptical Galaxies</h2>\n')
    these = (objtype == 'G') * (hubbletype <= 2)
    srt = np.argsort(hubbletype[these])
    html_rows(pngkeep[these][srt])

    html.write('<h2>Early-Type Disk Galaxies</h2>\n')
    these = (objtype == 'G') * (hubbletype >= 3) * (hubbletype <= 6)
    srt = np.argsort(hubbletype[these])
    html_rows(pngkeep[these][srt])

    html.write('<h2>Late-Type Disk Galaxies</h2>\n')
    these = (objtype == 'G') * (hubbletype >= 7) * (hubbletype <= 10)
    srt = np.argsort(hubbletype[these])
    html_rows(pngkeep[these][srt])
    
    html.write('<h2>Irregular Galaxies</h2>\n')
    these = (objtype == 'G') * (hubbletype >= 11)
    srt = np.argsort(hubbletype[these])
    html_rows(pngkeep[these][srt])
    
    html.write('</body></html>\n')

In [ ]:
if len(pngrej) > 0:
    with open(htmlfile_reject, 'w') as html:
        html.write('<html><head>\n')
        html.write('<style type="text/css">\n')
        html.write('img.ls-gallery {display: block;}\n')
        html.write('td.ls-gallery {width: 20%; word-wrap: break-word;}\n')
        html.write('</style>\n')
        html.write('</head><body>\n')
        html.write('<h1>DR6 Image Gallery - Rejected</h1>\n')
        html_rows(pngrej)
        html.write('</body></html>\n')